QCM-RF
=================


Setup
-----

First, we are going to import the required packages.

In [76]:

from __future__ import annotations

import contextlib
import json
from typing import TYPE_CHECKING, Callable

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

from qblox_instruments import Cluster, ClusterType

if TYPE_CHECKING:
    from numpy.typing import NDArray

    from qblox_instruments.qcodes_drivers.qcm_qrm import QcmQrm

### Scan For Clusters

We scan for the available devices connected via ethernet using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/api_reference/tools.html#api-pnp) for more info).

In [77]:
!qblox-pnp list

No devices found


In [78]:
cluster_ip = '192.0.2.141'
cluster_name = "cluster0"

### Connect to Cluster

We now make a connection with the Cluster.

In [34]:

# Close the chosen QCodes instrument to prevent name clash
with contextlib.suppress(KeyError):
    Cluster.find_instrument(cluster_name).close()

cluster = Cluster(
    name=cluster_name,
    identifier=cluster_ip,
    dummy_cfg={
        2: ClusterType.CLUSTER_QCM,
        4: ClusterType.CLUSTER_QRM,
        6: ClusterType.CLUSTER_QCM_RF,
        8: ClusterType.CLUSTER_QRM_RF,
    }
    if cluster_ip is None
    else None,
)

#### Get connected modules

In [35]:
def get_connected_modules(cluster: Cluster, filter_fn: Callable | None = None) -> dict[int, QcmQrm]:
    def checked_filter_fn(mod: ClusterType) -> bool:
        if filter_fn is not None:
            return filter_fn(mod)
        return True

    return {
        mod.slot_idx: mod for mod in cluster.modules if mod.present() and checked_filter_fn(mod)
    }

In [36]:
# QRM RF modules
modules = get_connected_modules(cluster, lambda mod: mod.is_qcm_type and mod.is_rf_type)
modules

{1: <QcmQrm: cluster0_module1 of Cluster: cluster0>,
 2: <QcmQrm: cluster0_module2 of Cluster: cluster0>,
 3: <QcmQrm: cluster0_module3 of Cluster: cluster0>,
 4: <QcmQrm: cluster0_module4 of Cluster: cluster0>,
 5: <QcmQrm: cluster0_module5 of Cluster: cluster0>,
 6: <QcmQrm: cluster0_module6 of Cluster: cluster0>,
 7: <QcmQrm: cluster0_module7 of Cluster: cluster0>,
 8: <QcmQrm: cluster0_module8 of Cluster: cluster0>,
 9: <QcmQrm: cluster0_module9 of Cluster: cluster0>,
 10: <QcmQrm: cluster0_module10 of Cluster: cluster0>,
 11: <QcmQrm: cluster0_module11 of Cluster: cluster0>,
 12: <QcmQrm: cluster0_module12 of Cluster: cluster0>,
 13: <QcmQrm: cluster0_module13 of Cluster: cluster0>}

In [44]:
readout_module = modules[11]
readout_module

<QcmQrm: cluster0_module11 of Cluster: cluster0>

### Reset the Cluster

We reset the Cluster to enter a well-defined state. Note that resetting will clear all stored parameters, so resetting between experiments is usually not desirable.

In [45]:
cluster.reset()
print(cluster.get_system_state())

Status: OKAY, Flags: NONE, Slot flags: NONE


In [46]:
# Parameters
no_averages = 10
integration_length = 1024
holdoff_length = 200
waveform_length = integration_length + holdoff_length

For simplicity, a single bin is used in this tutorial.

Now that the waveform and acquisition have been specified for the sequence, we need a Q1ASM program that sequences the waveforms and triggers the acquisitions. This program plays a DC wave, waits for the specified hold-off time and then acquires the signal. This process is repeated for the specified number of averages, with the average being done within the hardware.

In [73]:
seq_prog = f"""
loop:
      move    {no_averages},R0           # Average iterator.
      nop
    #   reset_ph
      
      set_awg_offs 10000, 10000
      upd_param 100 
      set_awg_offs  0,0  # Acquire data and store them in bin_n0 of acq_index.
      upd_param 500 
      loop     R0,@loop                  # Run until number of average iterations is done.
      stop                               # Stop the sequencer
      """

# Add sequence to single dictionary and write to JSON file.
sequence = {
    "waveforms": {},
    "weights": {},
    "acquisitions": {},
    "program": seq_prog,
}
with open("sequence.json", "w", encoding="utf-8") as file:
    json.dump(sequence, file, indent=4)
    file.close()

# Upload sequence
readout_module.sequencer0.sequence("sequence.json")

In [74]:
readout_module.disconnect_outputs()

# Configure channel map
readout_module.sequencer0.connect_sequencer("out0")

readout_module.sequencer0.marker_ovr_en(True)
readout_module.sequencer0.marker_ovr_value(3)  # Enables output on QRM-RF

# Set offset in mV
readout_module.out0_offset_path0(5.5)
readout_module.out0_offset_path1(5.5)



# Configure the sequencer
readout_module.sequencer0.mod_en_awg(True)
readout_module.sequencer0.nco_freq(500e6)
readout_module.sequencer0.sync_en(True)

# NCO delay compensation
readout_module.sequencer0.nco_prop_delay_comp_en(True)
readout_module.out0_lo_freq(3.9e9)

In [75]:



# Update the LO frequency.

readout_module.arm_sequencer(0)         
readout_module.start_sequencer() 



In [15]:
# Stop sequencer.
readout_module.stop_sequencer()

# Print status of sequencer.
print(readout_module.get_sequencer_state(0))

Status: STOPPED, Flags: FORCED_STOP


In [16]:
# Close the connection to the Cluster
cluster.close()